In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
# Original code
# spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Modified code with the online CSV link
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Calculate min and max payload for the range slider
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Get unique launch sites
launch_sites = spacex_df['Launch Site'].unique()

# Create a dash application
app = dash.Dash(__name__)

# Function to create dropdown options
def create_dropdown_options():
    return [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in launch_sites]

# Function to create app layout
def create_app_layout():
    return html.Div(children=[
        html.H1('SpaceX Launch Records Dashboard',
                style={'textAlign': 'center', 'color': '#503D36',
                       'font-size': 40}),

        # Modified dropdown with searchable attribute
        dcc.Dropdown(id='site-dropdown',
                     options=create_dropdown_options(),
                     value='ALL',
                     multi=False,
                     style={'width': '80%', 'margin-left': '10%'},
                     searchable=True,  # Added searchable attribute
                     placeholder="Select a Launch Site here"),  # Added placeholder attribute

        html.Div(dcc.Graph(id='success-pie-chart')),
        html.Br(),

        html.P("Payload range (Kg):"),

        # Modified range slider
        dcc.RangeSlider(id='payload-slider',
                        min=min_payload,
                        max=max_payload,
                        step=100,
                        marks={min_payload: f'{min_payload} kg', max_payload: f'{max_payload} kg'},
                        value=[min_payload, max_payload]),

        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ])

# Set the app layout
app.layout = create_app_layout()

# Set a default payload range for the slider
default_payload_range = [min_payload, max_payload]

# Callback to update the pie chart and scatter chart based on Launch Site selection and payload range
@app.callback([Output('success-pie-chart', 'figure'),
               Output('success-payload-scatter-chart', 'figure')],
              [Input('site-dropdown', 'value'),
               Input('payload-slider', 'value')])
def update_charts(selected_site, payload_range):
    # Filter data based on launch site selection
    if selected_site == 'ALL':
        filtered_data = spacex_df.copy()  # Use all data when 'ALL' is selected
    else:
        filtered_data = spacex_df[spacex_df['Launch Site'] == selected_site]

    # Filter data based on payload range
    filtered_data = filtered_data[
        (filtered_data['Payload Mass (kg)'] >= payload_range[0]) & (filtered_data['Payload Mass (kg)'] <= payload_range[1])
    ]

    # Update the pie chart accordingly
    pie_chart = px.pie(filtered_data, names='Launch Site', title=f'Successful Launches by Site - {selected_site}' if selected_site != 'ALL' else 'Total Successful Launches by Site')

    # Update the scatter chart based on payload correlation
    scatter_chart = px.scatter(filtered_data, x='Payload Mass (kg)', y='class',
                               color='Booster Version Category',  # Color-label based on Booster Version Category
                               labels={'class': 'Launch Outcome'},
                               title='Correlation between Payload and Launch Success')

    return pie_chart, scatter_chart

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
